In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt

dataset_path = "../datasets/1.0v/"

!ls ../datasets/1.0v

## Loading our datasets...

infos = pd.read_csv(dataset_path + 'infos.csv', sep='|')
items = pd.read_csv(dataset_path + 'items.csv', sep='|')
orders = pd.read_csv(dataset_path + 'orders.csv', sep='|')

Features.pdf  infos.csv  items.csv  orders.csv	README.md  Task_2020.pdf


### Removing the items that were never sold

In [2]:
# Taking the ids of the items that were sold at least once...
sold_items_id = orders['itemID'].unique()
sold_items_id

array([ 450,   83, 7851, ..., 6427, 8604, 9796])

In [3]:
# Items that were sold at least once will be stored in 'sold_items'
sold_items = items[items.itemID.isin(sold_items_id)]
sold_items

,itemID,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
0,1,0,1,4.38,1,1,1,8.84
1,2,0,2,3.00,1,2,1,16.92
2,3,0,3,5.00,1,3,1,15.89
3,4,0,2,4.44,1,2,1,40.17
4,5,0,2,2.33,1,1,1,17.04
...,...,...,...,...,...,...,...,...
10449,10450,182,227,0.00,8,44,8,36.78
10458,10459,180,253,0.00,8,44,8,56.57
10459,10460,0,253,0.00,8,44,8,163.81
10461,10462,180,253,0.00,8,44,8,166.97


Just making a sanity check to be sure if we really have kept only the items that had been sold at least once...

In [4]:
fact_check = np.array(sold_items.itemID)
fact_check == sorted(sold_items_id)

array([ True,  True,  True, ...,  True,  True,  True])

In [5]:
# We'll extract only the infos about the items that were sold at least once...
sold_infos = infos[infos.itemID.isin(sold_items_id)]
sold_infos

,itemID,simulationPrice,promotion
0,1,3.43,NaN
1,2,9.15,NaN
2,3,14.04,NaN
3,4,14.10,NaN
4,5,7.48,NaN
...,...,...,...
10449,10450,44.43,2018-07-10
10458,10459,14.71,NaN
10459,10460,325.67,NaN
10461,10462,304.30,NaN


Another sanity check...

In [6]:
fact_check_2 = np.array(sold_infos.itemID)
fact_check_2 == sorted(sold_items_id)

array([ True,  True,  True, ...,  True,  True,  True])

### <span style="color:red">Most revenue items</span>

It's very strange that some items are sold below the 'recommended retail price', even for promotion weeks... This might be useful, since the weeks we need to predict have promotions scheduled for them...

In [7]:
# Concatenating the itemID column with the revenue column...
sold_items_revenue = pd.concat([sold_items.itemID, sold_infos.simulationPrice -  sold_items.recommendedRetailPrice], axis=1, names=['itemID', 'revenue'])
sold_items_revenue

,itemID,0
0,1,-5.41
1,2,-7.77
2,3,-1.85
3,4,-26.07
4,5,-9.56
...,...,...
10449,10450,7.65
10458,10459,-41.86
10459,10460,161.86
10461,10462,137.33
